In [10]:
import h5py 
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import load_model
from keras import optimizers
import sys,pdb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


f = h5py.File('binary_random_sp2019.hdf5', 'r') #small dataset, assume this our full data
print('keys of the dataset: ', list(f.keys())) #useful for seeing the data set name/key

keys of the dataset:  ['human', 'machine']


In [11]:
xhuman = f['human'][:]
xmachine = f['machine'][:]
yhuman=np.ones((xhuman.shape[0],1),int)
ymachine=np.zeros((xhuman.shape[0],1),int)
xdata=np.concatenate((xhuman,xmachine), axis=0)
ydata=np.concatenate((yhuman,ymachine), axis=0)
xtrain,xtest,ytrain,ytest=train_test_split(xdata,ydata,test_size=0.2)

In [12]:
# Define a functional API model
# Layer 1:                  20 * 20
# Activation 1:             relu
# Layer 2:                  20 * 1
# Activation 2:             logistic-sigmoid, binary cross-entropy loss
############################################################
# This returns a tensor
inputs = Input(shape=(20,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(20, activation='relu')(inputs)
predictions = Dense(1, activation='sigmoid')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
print(model.summary())

############################################################
# Model compilation
# Must be called before starting training
############################################################
# For a multi-class classification problem
opt=optimizers.SGD(lr=0.1,momentum=0,decay=0,nesterov=True)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 20)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 21        
Total params: 441
Trainable params: 441
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
# Model training
############################################################
# if y_train is not categorical, you may use this 
# or, write your own code to convert
#one_hot_labels = keras.utils.to_categorical(labels, num_classes=2)

#model.fit(xtrain, ytrain, epochs=10, batch_size=16)
model.fit(xtrain, ytrain, epochs=2000, batch_size=16,validation_data=([xtest],[ytest]))

Train on 3840 samples, validate on 960 samples
Epoch 1/2000


InternalError: GPU sync failed

In [56]:
list(model.history.history)
val_loss=model.history.history['val_loss']
val_acc=model.history.history['val_acc']
loss=model.history.history['loss']
acc=model.history.history['acc']

In [ ]:
# plot figure 
fig, ax = plt.subplots()
ax.plot(val_acc,'r',label='validation')
ax.plot(acc,'b',label='training')

ax.set(xlabel='number of training epoch', ylabel='predict accuracy',
       title='test vs. validation accuracy () with different number of epoch')
ax.grid()
ax.legend()

fig.savefig("learning curve.png")
plt.show()


accuracy


In [58]:
score = model.evaluate(xtest, ytest, batch_size=16)
print('Test Loss = ', score[0])
print('Test Accuracy = ', score[1])

960/960 [==============================] - 0s 125us/step
Test Loss =  0.6853818838795026
Test Accuracy =  0.6260416666666667


In [59]:
model.save('my_P3model.h5')